## Code to search the contents for all of your SO answers

- File with
  - Tags
  - Title
  - ViewCount
  - Score
  - Answer ID
  - Accepted Answer Id
  - Answer Body

In [ ]:
import pandas as pd
from IPython.display import HTML
from bs4 import BeautifulSoup

In [ ]:
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

- Perform query at [StackExchange Data Explorer][sede] with the following query, using your Stack Overflow user ID

```sql
SELECT 
    Ans.CreationDate, Qus.Tags, Qus.Title, Qus.ViewCount, Qus.Body AS QBody,
    Ans.Score, Ans.ID, Qus.AcceptedAnswerId, Ans.Body
FROM Posts as Qus
LEFT JOIN Posts as Ans
ON Qus.ID = Ans.ParentID
WHERE Ans.OwnerUserId = ##UserId##
ORDER BY Ans.CreationDate DESC
```

- Save the file as `'so_QueryResults.csv'`

  [sede]: https://data.stackexchange.com/stackoverflow/query/new

In [ ]:
# load file
so = pd.read_csv('so_QueryResults.csv', dtype={'ID': int, 'AcceptedAnswerId': str})

In [ ]:
so.CreationDate = pd.to_datetime(so.CreationDate).dt.date

In [ ]:
so.AcceptedAnswerId.fillna(value=0, inplace=True)  # replace NaN with 0
so.AcceptedAnswerId = so.AcceptedAnswerId.astype(int)  # convert from float to int
so['Accepted'] = so.ID == so.AcceptedAnswerId  # add a Boolean column True if your answer is the accepted answer

In [ ]:
# using the answer id, create a link to your answer
so['Answer Link'] = so[['ID', 'Title']].apply(lambda x: f'<a href="https://stackoverflow.com/questions/{x[0]}">{x[1]}</a>', axis=1)

# add Answer Link to the answer body as a header
so['Body'] = so[['Answer Link', 'Body']].apply(lambda x: f'<h2>{x[0]}</h2>' + x[1], axis=1)

In [ ]:
so.drop(columns=['ID', 'AcceptedAnswerId', 'Title'], inplace=True)

In [ ]:
so.Tags = so.Tags.str.replace('><', ' ', regex=True).str.replace('<|>', '', regex=True).str.split()

# replace newline with html newline tag
so.QBody = so.QBody.str.replace("\\n+", "<br>", regex=True)
so.Body = so.Body.str.replace("\\n+", "<br>", regex=True)

In [ ]:
so.head()

In [ ]:
so.tail()

In [ ]:
# sort by ViewCount
# so = so.sort_values(['ViewCount'], ascending=False)  # ['Accepted', 'Score', 'ViewCount']  
# so.head(100)

In [ ]:
# sort by Accepted and CreationDate
# so = so.sort_values(['Accepted', 'CreationDate'], ascending=False).reset_index(drop=True)
# so.head(100)

In [ ]:
# various filters for search the Body of the answer
son = so[so.Body.str.lower().str.contains('bar_label', regex=False)]
# son = son[son.Body.str.lower().str.contains('year', regex=False)]
# son = son[son.Body.str.lower().str.contains('.plot', regex=False)]
# son = son[~son.Body.str.lower().str.contains('bar', regex=False)]
son

In [ ]:
so = so.drop(columns=['Body'])  # drop Body of the html will be a mess

In [ ]:
so.to_html('../trenton3983.github.io/files/so_answers_new.html', escape=False)  # html to a file

In [ ]:
so.to_html('so_answers_body.html', columns=['QBody', 'Body'], justify='left', index=False, escape=False)

In [ ]:
HTML(so.head(1).to_html(escape=False, columns=['Body']).replace("\\n","<br>"))  # html displayed in notebook

Header to add to html file. See [CSS Table Alignment][1] and [Try It][2]

```html
<head>
<style>
table {
  table-layout: fixed;
  width: 2400px;
}

td, th {
  width: 1200px;
  vertical-align: top;
  overflow: wrap;
}
</style>
</head>
```

 [1]: https://www.w3schools.com/css/css_table_align.asp
 [2]: https://www.w3schools.com/css/tryit.asp?filename=trycss_table_vertical-align

## Print Title and Link

- For adding links into SO answer

In [ ]:
so['Link'] = so['ID'].apply(lambda x: f"https://stackoverflow.com/questions/{x}")

In [ ]:
so.head()

In [ ]:
so.iloc[0, -1]

In [ ]:
son = so[so.Body.str.lower().str.contains('bar_label', regex=False)]

In [ ]:
for i, v in enumerate(son.Title, 1):
    print(f'- [{v}][ex{i}]')
    
for i, v in enumerate(son.Link, 1):
    print(f'  [ex{i}]: {v}')